In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/smiles-extra-data/data_dnst1.xlsx
/kaggle/input/smiles-extra-data/data_tg3.xlsx
/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv
/kaggle/input/open-polymer-challenge/sample_submission.csv
/kaggle/input/open-polymer-challenge/train.csv
/kaggle/input/open-polymer-challenge/test.csv
/kaggle/input/open-polymer-challenge/train_supplement/dataset2.csv
/kaggle/input/open-polymer-challenge/train_supplement/dataset4.csv
/kaggle/input/open-polymer-challenge/train_supplement/dataset1.csv
/kaggle/input/open-polymer-challenge/train_supplement/dataset3.csv
/kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/config.json
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/merges.txt
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/tokenizer.json
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/vocab.json
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/tokenizer

In [2]:
!cp -r /kaggle/input/autogluon-package/* /kaggle/working/
!pip install -f --quiet --no-index --find-links='/kaggle/input/autogluon-package' 'autogluon.tabular-1.3.1-py3-none-any.whl'

Looking in links: --quiet, /kaggle/input/autogluon-package
Processing ./autogluon.tabular-1.3.1-py3-none-any.whl
Processing /kaggle/input/autogluon-package/scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from autogluon.tabular==1.3.1)
Processing /kaggle/input/autogluon-package/autogluon.core-1.3.1-py3-none-any.whl (from autogluon.tabular==1.3.1)
Processing /kaggle/input/autogluon-package/autogluon.features-1.3.1-py3-none-any.whl (from autogluon.tabular==1.3.1)
Processing /kaggle/input/autogluon-package/autogluon.common-1.3.1-py3-none-any.whl (from autogluon.core==1.3.1->autogluon.tabular==1.3.1)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encode

In [3]:
!cp -r /kaggle/input/scikit-package/* /kaggle/working/
!pip install -f --quiet --no-index --find-links='/kaggle/input/scikit-package' 'scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl' 


Looking in links: --quiet, /kaggle/input/scikit-package
Processing ./scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [5]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

import pandas as pd

csv_path = '/kaggle/input/open-polymer-challenge/train.csv'
train_df = pd.read_csv(csv_path)
csv2_path = '/kaggle/input/open-polymer-challenge/test.csv'
test_df = pd.read_csv(csv2_path)

In [7]:
# 外部データ読み込み

# https://www.kaggle.com/datasets/minatoyukinaxlisa/tc-smiles
data_tc = pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv')
data_tc = data_tc.rename(columns={'TC_mean': 'Tc'})

# https://springernature.figshare.com/articles/dataset/dataset_with_glass_transition_temperature/24219958?file=42507037
data_tg2 = pd.read_csv('/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv', usecols=['SMILES', 'Tg (C)'])
data_tg2 = data_tg2.rename(columns={'Tg (C)': 'Tg'})

# https://www.sciencedirect.com/science/article/pii/S2590159123000377#ec0005
data_tg3 = pd.read_excel('/kaggle/input/smiles-extra-data/data_tg3.xlsx')
data_tg3 = data_tg3.rename(columns={'Tg [K]': 'Tg'})
data_tg3['Tg'] = data_tg3['Tg'] - 273.15

# https://github.com/Duke-MatSci/ChemProps
data_dnst = pd.read_excel('/kaggle/input/smiles-extra-data/data_dnst1.xlsx')
data_dnst = data_dnst.rename(columns={'density(g/cm3)': 'Density'})[['SMILES', 'Density']]


In [8]:
# 列が足りない場合には NaN を追加して結合します
combined_df = pd.concat([train_df, data_tc, data_tg2, data_tg3, data_dnst], axis=0, ignore_index=True)

In [9]:
train_df = combined_df
train_df['Density'] = pd.to_numeric(train_df['Density'], errors='coerce')

In [10]:
from rdkit import Chem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors, rdmolops
import networkx as nx
import pandas as pd
from tqdm import tqdm
import numpy as np

# === 1. 208個のdescriptor名を取得 ===
descriptor_names = [desc[0] for desc in Descriptors._descList]

# === 2. descriptor計算器を準備 ===
calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

# === 3. 記述子計算関数（例外をnp.nanで吸収） ===
def compute_descriptors(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return [np.nan] * len(descriptor_names)
        return calc.CalcDescriptors(mol)
    except Exception as e:
        print(f"⚠️ SMILES: {smiles} -> 記述子計算エラー: {e}")
        return [np.nan] * len(descriptor_names)

# === 4. グラフ特徴量計算関数（安全処理あり） ===
def compute_graph_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return [np.nan, np.nan, np.nan]
        adj = rdmolops.GetAdjacencyMatrix(mol)
        G = nx.from_numpy_array(adj)
        diameter = nx.diameter(G) if nx.is_connected(G) else 0
        avg_shortest = nx.average_shortest_path_length(G) if nx.is_connected(G) else 0
        num_cycles = len(list(nx.cycle_basis(G)))
        return [diameter, avg_shortest, num_cycles]
    except Exception as e:
        print(f"⚠️ SMILES: {smiles} -> グラフ特徴量計算エラー: {e}")
        return [np.nan, np.nan, np.nan]

# === 5. SMILES列から 208記述子 + グラフ特徴量 を計算 ===
descriptors_list = []
graph_feats_list = []

for smi in tqdm(train_df['SMILES'], desc="記述子＋グラフ特徴量計算"):
    descriptors_list.append(compute_descriptors(smi))
    graph_feats_list.append(compute_graph_features(smi))

# === 6. DataFrame化 ===
descriptor_df = pd.DataFrame(descriptors_list, columns=descriptor_names)
graph_feats_df = pd.DataFrame(graph_feats_list, columns=['graph_diameter', 'avg_shortest_path', 'num_cycles'])

# === 7. 結合（元のtrain_dfと） ===
train_df_with_desc = pd.concat([train_df.reset_index(drop=True), descriptor_df, graph_feats_df], axis=1)

# === 8. 欠損確認 ===
print("✅ 欠損確認 (train_df_with_desc)")
print(train_df_with_desc.isnull().sum().sort_values(ascending=False).head())

記述子＋グラフ特徴量計算: 100%|█████████▉| 10754/10797 [04:00<00:00, 97.04it/s] [16:11:44] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[16:11:44] SMILES Parse Error: check for mistakes around position 12:
[16:11:44] *O[Si](*)([R])[R]
[16:11:44] ~~~~~~~~~~~^
[16:11:44] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[16:11:44] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[16:11:44] SMILES Parse Error: check for mistakes around position 12:
[16:11:44] *O[Si](*)([R])[R]
[16:11:44] ~~~~~~~~~~~^
[16:11:44] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[16:11:44] SMILES Parse Error: syntax error while parsing: *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
[16:11:44] SMILES Parse Error: check for mistakes around position 28:
[16:11:44] c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=
[16:11:44] ~~~~~~~~~~~~~~~~~~~~^
[16:11:44] SMILES Parse Error: Failed parsing SMILES '

✅ 欠損確認 (train_df_with_desc)
Rg               10183
BCUT2D_MRHI       9984
BCUT2D_LOGPHI     9984
BCUT2D_MWHI       9984
BCUT2D_MWLOW      9984
dtype: int64


In [11]:
from tqdm import tqdm

# === test の記述子計算＋グラフ特徴量計算 ===
test_descriptors_list = []
test_graph_feats_list = []

for smi in tqdm(test_df['SMILES'], desc="test 記述子＋グラフ特徴量計算"):
    test_descriptors_list.append(compute_descriptors(smi))
    test_graph_feats_list.append(compute_graph_features(smi))

# === DataFrame化 ===
test_descriptor_df = pd.DataFrame(test_descriptors_list, columns=descriptor_names)
test_graph_feats_df = pd.DataFrame(test_graph_feats_list, columns=['graph_diameter', 'avg_shortest_path', 'num_cycles'])

# === test_dfと連結 ===
test_df_with_desc = pd.concat([test_df.reset_index(drop=True), test_descriptor_df, test_graph_feats_df], axis=1)

# === train で定数列（全て同じ値 or 全欠損）を検出 ===
constant_columns = []
for col in train_df_with_desc.columns:
    try:
        nunique = train_df_with_desc[col].nunique(dropna=False)
        all_null = train_df_with_desc[col].isnull().all()
        if nunique == 1 or all_null:
            constant_columns.append(col)
    except Exception as e:
        print(f"列 {col} の処理でエラー: {e}")
        continue

print(f"定数または全欠損列の数: {len(constant_columns)}")
print("例:", constant_columns[:10])

# === train, test から削除 ===
train_df_with_desc = train_df_with_desc.drop(columns=constant_columns, errors='ignore')
test_df_with_desc = test_df_with_desc.drop(columns=constant_columns, errors='ignore')

# === 欠損確認 ===
print(test_df_with_desc.isnull().sum().sort_values(ascending=False).head())

test 記述子＋グラフ特徴量計算: 100%|██████████| 3/3 [00:00<00:00, 32.18it/s]

定数または全欠損列の数: 0
例: []
BCUT2D_MRLOW     3
BCUT2D_CHGHI     3
BCUT2D_MWHI      3
BCUT2D_CHGLO     3
BCUT2D_LOGPHI    3
dtype: int64


In [12]:
# 1. 欠損率が高い列を除去（例: 20%以上のNaN）
nan_threshold = 0.2  # 20%を閾値に設定

# train側の記述子だけで欠損率計算
desc_df = train_df_with_desc.loc[:, descriptor_names]  # 記述子のみ抽出
nan_frac = desc_df.isnull().mean()

# 欠損率20%以上の列は除外
cols_to_keep = nan_frac[nan_frac < nan_threshold].index.tolist()

print(f"欠損率20%以上の列を除外: {len(desc_df.columns) - len(cols_to_keep)} 列")

# 2. 定数列（全て同じ値の列）を除去（trainとtest両方で）

constant_columns = []
for col in cols_to_keep:
    if train_df_with_desc[col].nunique(dropna=False) == 1:
        constant_columns.append(col)

print(f"定数列の数: {len(constant_columns)}")

# 除去対象の列
cols_filtered = [col for col in cols_to_keep if col not in constant_columns]

# train/test 両方から該当列を除去
train_df_with_desc = train_df_with_desc.drop(columns=[col for col in desc_df.columns if col not in cols_filtered], errors='ignore')
test_df_with_desc = test_df_with_desc.drop(columns=[col for col in desc_df.columns if col not in cols_filtered], errors='ignore')

print(f"最終的に使用する記述子数: {len(cols_filtered)}")

欠損率20%以上の列を除外: 12 列
定数列の数: 0
最終的に使用する記述子数: 205


In [13]:
# 1. train_df_with_desc に実際に存在する descriptor のみ使用し、80%以上 NaN の列を除外
existing_descriptor_names = [name for name in descriptor_names if name in train_df_with_desc.columns]
desc_df = train_df_with_desc[existing_descriptor_names]
valid_descriptor_mask = desc_df.isnull().mean() < 0.2
filtered_descriptor_names = desc_df.columns[valid_descriptor_mask].tolist()
print(f"使用する記述子数: {len(filtered_descriptor_names)} / {len(descriptor_names)}")

使用する記述子数: 205 / 217


In [14]:
# filtered_descriptor_namesに対してinfをNaNに置換し、NaNは中央値補完
for col in filtered_descriptor_names:
    # infをNaNに置換
    train_df_with_desc[col] = train_df_with_desc[col].replace([np.inf, -np.inf], np.nan)
    test_df_with_desc[col] = test_df_with_desc[col].replace([np.inf, -np.inf], np.nan)
    
    # 極端に大きな値もNaNに
    max_float32 = np.finfo(np.float32).max
    train_df_with_desc[col] = train_df_with_desc[col].apply(
        lambda x: np.nan if isinstance(x, (int, float)) and abs(x) > max_float32 else x
    )
    test_df_with_desc[col] = test_df_with_desc[col].apply(
        lambda x: np.nan if isinstance(x, (int, float)) and abs(x) > max_float32 else x
    )

# 中央値でNaN補完（学習時のみに）
median_values = train_df_with_desc[filtered_descriptor_names].median()
train_df_with_desc[filtered_descriptor_names] = train_df_with_desc[filtered_descriptor_names].fillna(median_values)
test_df_with_desc[filtered_descriptor_names] = test_df_with_desc[filtered_descriptor_names].fillna(median_values)

/tmp/ipykernel_13/2572210559.py:10: RuntimeWarning: invalid value encountered in less
  lambda x: np.nan if isinstance(x, (int, float)) and abs(x) > max_float32 else x


In [15]:
import numpy as np
from autogluon.tabular import TabularPredictor

models = {}

def predict_test_autogluon(df, predictor, target_col, feature_cols):
    # AutoGluonのpredictorで予測し、dfに代入する関数
    # feature_colsがdfに必須なので事前に確認してください
    y_pred = predictor.predict(df[feature_cols])
    df.loc[df.index, target_col] = y_pred
    return df

def fill_with_autogluon(df, target, descriptors, time_limit=3600):
    import numpy as np
    from autogluon.tabular import TabularPredictor

    target_series = df[target]
    non_null_mask = target_series.notnull()

    train_data = df.loc[non_null_mask, descriptors + [target]].copy()
    # infをnanに変換
    train_data = train_data.replace([np.inf, -np.inf], np.nan)

    # 欠損値があれば中央値で補完
    if train_data.isnull().any().any():
        median_values = train_data.median()
        train_data = train_data.fillna(median_values)

    # ここで再度train_dataにnanがあるかチェック
    if train_data.isnull().any().any():
        print(f"[{target}] 補完後もNaNあり。スキップ。")
        return df, None

    pred_data = df.loc[~non_null_mask, descriptors].copy()
    pred_data = pred_data.replace([np.inf, -np.inf], np.nan)

    # 予測用データも同様に欠損補完（中央値）
    if pred_data.isnull().any().any():
        median_values = pred_data.median()
        pred_data = pred_data.fillna(median_values)

    if train_data.empty or pred_data.empty:
        print(f"[{target}] 有効な学習/予測データなし。スキップ。")
        return df, None

    predictor = TabularPredictor(label=target, problem_type='regression').fit(
    train_data=train_data,
    time_limit=time_limit,
    presets='best_quality',
    excluded_model_types=['KNN'],  # 不要であれば
    verbosity=2
    )

    try:
        y_pred = predictor.predict(pred_data)
        df.loc[pred_data.index, target] = y_pred
        print(f"[{target}] 欠損 {len(y_pred)} 件を AutoGluon で補完")
    except ValueError as e:
        print(f"[{target}] ❌予測失敗: {e}")

    return df, predictor

In [16]:
# FFV補完（記述子のみ）
train_df_with_desc, ffv_model = fill_with_autogluon(
    train_df_with_desc,
    target='FFV',
    descriptors=filtered_descriptor_names,
    time_limit=120
)
models['FFV'] = ffv_model

# 学習済みモデルのパフォーマンス確認
if ffv_model is not None:
    leaderboard_df = ffv_model.leaderboard(silent=True)
    print(leaderboard_df)

# FFV列のNaN・inf・overflowを除去
train_df_with_desc['FFV'] = train_df_with_desc['FFV'].replace([np.inf, -np.inf], np.nan)
train_df_with_desc['FFV'] = train_df_with_desc['FFV'].apply(
    lambda x: np.nan if isinstance(x, (int, float)) and abs(x) > np.finfo(np.float32).max else x
)

# 他ターゲット補完（FFV含む）
extended_features = filtered_descriptor_names + ['FFV']
for target in ['Tg', 'Tc', 'Density', 'Rg']:
    train_df_with_desc, model = fill_with_autogluon(
        train_df_with_desc,
        target=target,
        descriptors=extended_features,
        time_limit=3600
    )
    models[target] = model

No path specified. Models will be saved in: "AutogluonModels/ag-20250914_161152"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.09 GB / 31.35 GB (96.0%)
Disk Space Avail:   15.94 GB / 19.52 GB (81.7%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be 

[FFV] 欠損 3767 件を AutoGluon で補完
                 model  score_val              eval_metric  pred_time_val  \
0    LightGBMXT_BAG_L1  -0.013862  root_mean_squared_error       0.202630   
1  WeightedEnsemble_L2  -0.013862  root_mean_squared_error       0.203134   
2  WeightedEnsemble_L3  -0.013862  root_mean_squared_error       0.203200   
3    LightGBMXT_BAG_L2  -0.014225  root_mean_squared_error       0.276971   

    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  \
0  20.266397                0.202630          20.266397            1   
1  20.269587                0.000504           0.003190            2   
2  20.275077                0.000569           0.008681            3   
3  30.261060                0.074341           9.994664            2   

   can_infer  fit_order  
0       True          1  
1       True          2  
2       True          4  
3       True          3  


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 18 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 8): ['SMR_VSA8', 'SlogP_VSA9', 'fr_isocyan', 'fr_isothiocyan', 'fr_nitroso', 'fr_prisulfonamd', 'fr_tetrazole', 'fr_thiocyan']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 7): ['MaxEStateIndex', 'fr_COO2', 'fr_Nhpyrrole', 'fr_amide', 'fr_diazo', 'fr_phenol', 'fr_phos_ester']
		These features were not used to generate any of the output features. Add a feature generator compati

[1000]	valid_set's rmse: 33.4509
[2000]	valid_set's rmse: 33.047
[3000]	valid_set's rmse: 33.0122
[4000]	valid_set's rmse: 32.9911
[5000]	valid_set's rmse: 32.9865
[6000]	valid_set's rmse: 32.9834
[7000]	valid_set's rmse: 32.9823
[8000]	valid_set's rmse: 32.9824
[9000]	valid_set's rmse: 32.9823
[1000]	valid_set's rmse: 35.8124
[2000]	valid_set's rmse: 35.651
[3000]	valid_set's rmse: 35.6416
[1000]	valid_set's rmse: 39.9453
[2000]	valid_set's rmse: 39.7677
[1000]	valid_set's rmse: 33.5928
[1000]	valid_set's rmse: 30.7804
[1000]	valid_set's rmse: 39.0459
[2000]	valid_set's rmse: 38.6394
[3000]	valid_set's rmse: 38.5989
[1000]	valid_set's rmse: 41.9016
[2000]	valid_set's rmse: 41.4092
[3000]	valid_set's rmse: 41.3295
[4000]	valid_set's rmse: 41.3058
[5000]	valid_set's rmse: 41.2966
[6000]	valid_set's rmse: 41.2893
[7000]	valid_set's rmse: 41.2868
[8000]	valid_set's rmse: 41.2853
[9000]	valid_set's rmse: 41.2839
[10000]	valid_set's rmse: 41.2832
[1000]	valid_set's rmse: 39.8521


	-36.7134	 = Validation score   (-root_mean_squared_error)
	90.5s	 = Training   runtime
	0.57s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 505.51s of the 804.99s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


...

[1000]	valid_set's rmse: 2.55093
[2000]	valid_set's rmse: 2.53429
[3000]	valid_set's rmse: 2.53015
[4000]	valid_set's rmse: 2.53155


	-2.6221	 = Validation score   (-root_mean_squared_error)
	21.29s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetFastAI_r191_BAG_L2 ... Training model for up to 637.03s of the 636.95s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
No improvement since epoch 6: early stopping
No improvement since epoch 10: early stopping
No improvement since epoch 9: early stopping
No improvement since epoch 22: early stopping
	-2.9151	 = Validation score   (-root_mean_squared_error)
	14.87s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost_r9_BAG_L2 ... Training model for up to 621.91s of the 621.83s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-2.591	 = Validation score   (-root_mean_squared_error)
	198.04s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: LightGBM_r96_BAG_L2 ... Training model for up to 423.69s 

[1000]	valid_set's rmse: 2.89108
[2000]	valid_set's rmse: 2.89074
[3000]	valid_set's rmse: 2.89073


	-2.6131	 = Validation score   (-root_mean_squared_error)
	15.57s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI_r145_BAG_L2 ... Training model for up to 80.14s of the 80.06s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
No improvement since epoch 4: early stopping
No improvement since epoch 4: early stopping
No improvement since epoch 4: early stopping
No improvement since epoch 5: early stopping
	-2.8685	 = Validation score   (-root_mean_squared_error)
	12.86s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: XGBoost_r89_BAG_L2 ... Training model for up to 67.03s of the 66.96s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-2.5855	 = Validation score   (-root_mean_squared_error)
	12.8s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetTorch_r30_BAG_L2 ... Training model for up to 54.07s of

[Rg] 欠損 10183 件を AutoGluon で補完


In [17]:
# FFV予測
test_df_with_desc = predict_test_autogluon(
    test_df_with_desc,
    models['FFV'],
    'FFV',
    filtered_descriptor_names
)

# 他ターゲット予測
for target in ['Tg', 'Tc', 'Density', 'Rg']:
    test_df_with_desc = predict_test_autogluon(
        test_df_with_desc,
        models[target],
        target,
        extended_features
    )

# 提出ファイル作成
targets = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
submission = test_df_with_desc[['id'] + targets].copy()
submission.to_csv('submission.csv', index=False)

print("✅ 提出用ファイル 'submission.csv' を作成しました。")
print(submission.head())

/tmp/ipykernel_13/2844734425.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[df.index, target_col] = y_pred
/tmp/ipykernel_13/2844734425.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[df.index, target_col] = y_pred
/tmp/ipykernel_13/2844734425.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfra

✅ 提出用ファイル 'submission.csv' を作成しました。
           id          Tg       FFV        Tc   Density         Rg
0  1109053969  163.082565  0.377299  0.163123  1.180020  21.536901
1  1422188626  201.277557  0.379842  0.241983  1.102160  20.952915
2  2032016830   98.791893  0.353015  0.257450  1.126908  21.392076


/tmp/ipykernel_13/2844734425.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[df.index, target_col] = y_pred
